In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
import random
import os
from scipy.stats import norm

In [ ]:
dir='/Users/reneehlozek/Code/CIFAR_Network/GIMME/GIMME_analyses/CleanedGIMMEData/'
num='01'
data = np.loadtxt(dir+'Control_group_output/individual/'+'10'+num+'Betas.csv', skiprows=1, usecols=range(1,9), delimiter=',')
A = data[:,4:] #same day beta values, (4x4)
B = data[:,:4] #lagged beta values
VR, SS, ANX, INT = np.genfromtxt(dir+'Control_Group/10'+num+'.txt', skip_header=1, unpack=True)
length=75

var_vr, mean_vr = np.nanvar(VR), np.nanmean(VR)  
var_ss, mean_ss = np.nanvar(SS), np.nanmean(SS)
var_anx, mean_anx = np.nanvar(ANX), np.nanmean(ANX)
var_int, mean_int = np.nanvar(INT), np.nanmean(INT) 

## Indicator matrix equation 
We want to compute the value of the indicators (combined into a vector $M$) as a function of contemporaneous matrix A and lagged matrix B for each person. This simplifies to:
$M[t] = (I-A)^{-1}BM[t-1] + C$

Note that the multiplication is matrix multiplication, we can't use dot products here.

In [ ]:
M = np.zeros((length,4))
t = np.arange(0,length) 
#pick our starting M
C = np.array([var_vr, var_ss,var_anx,var_int])
M[0,:] = np.array([mean_vr, mean_ss, mean_anx, mean_int])+ np.random.randn(4)*C
# Compute our covariance matrix C from the std of the chains
print(C)
# now for each day, we compute the matrix multiplication from the day before
for r in range(1,np.shape(M)[0]):   
    M[r]=np.matmul(np.matmul(np.linalg.inv(np.eye(4)-A),B),M[r-1])     #+ np.matmul(np.linalg.inv(np.eye(4)-A),np.dot(np.random.randn(4),C)) + np.array([mean_vr, mean_ss, mean_anx, mean_int])
    M[r]+=np.random.randn(4)*C

M[:,:]+=  np.array([mean_vr, mean_ss, mean_anx, mean_int])
print(M[0,1])
plt.figure(3)
plt.plot(t,VR,  color='purple',label='VR data', linestyle='--')
plt.plot(t, SS, color='teal',label='SS data', linestyle='--')
plt.plot(t,ANX, color='orange',label='ANX data', linestyle='--')
plt.plot(t,INT, color='gray', label='INT data', linestyle='--')
leg=plt.legend()
print('var sim', 'var_data')
print(np.var(M[:,0]),var_vr, 'VR')
print(np.var(M[:,1]),var_ss, 'SS')
print(np.var(M[:,2]),var_anx, 'ANX')
print(np.var(M[:,3]),var_int, 'INT')

In [ ]:
#clipping VR (0-5)

indices = np.where(M[:,0]<0)[0]
M[indices,0] = 0
indices = np.where(M[:,0]>5)[0]
M[indices,0] = 5

for i in range(1,4):
#clipping other variables (1-5)
    indices = np.where(M[:,i]<1)[0]
    M[indices,i] = 1
    indices = np.where(M[:,i]>5)[0]
    M[indices,i] = 5

In [ ]:
plt.figure(2, figsize=(10,6))


plt.plot(t,M[:,0], color='purple',label='VR sim')
plt.plot(t,VR,  color='purple',label='VR data', linestyle='--')
plt.plot(t,M[:,1], color='teal',label='SS sim')
plt.plot(t, SS, color='teal',label='SS data', linestyle='--')
leg=plt.legend()
plt.figure(3, figsize=(10,6))
plt.plot(t,M[:,2], color='orange',label='ANX sim')
plt.plot(t,ANX, color='orange',label='ANX data', linestyle='--')
plt.plot(t,M[:,3], color='gray', label='INT sim')
plt.plot(t,INT, color='gray', label='INT data', linestyle='--')
leg=plt.legend()